In [1]:
import sys
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix

sys.path.append('../../')
import poe_price

pd.set_option('display.max_columns', None)

# Data

In [4]:
(name, params), = config['select']['items'].items()
items = eval(name)(**params)

(name, params), = config['select']['currencies'].items()
currencies = eval(name)(**params)

AttributeError: module 'poe_price.data' has no attribute 'select'

# Features extraction

In [ ]:


# generate features
X = []
for v in config['feature']['X']:
    (name, params), = v.items()
    name = eval(name)
    if isfunction(name):
        feature = name(items, **params)
    elif isclass(name):
        name = name(**params)
        feature = name.fit_transform(items)
    X.append(feature if issparse(feature) else csr_matrix(feature))
X = hstack(X)

(name, params), = config['feature']['y'].items()
name = eval(name)
if isfunction(name):
    y = name(items, **params)
elif isclass(name):
    name = name(**params)
    y = name.fit_transform(items, currencies)

# preprocess (remove null features, standardize / , split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,
                                                    random_state = 0)

# train model
(name, params), = config['model'].items()
name = eval(name)
print(name)
regressor = name(**params)
regressor.fit(X_train, y_train)
y_train_pred = regressor.predict(X_train)
y_test_pred = regressor.predict(X_test)

print(r2_score(y_train, y_train_pred), r2_score(y_test, y_test_pred))